# 双重差分 (Difference-in-Differences) - 政策评估的利器

---

## 学习目标

通过本章学习，你将能够：

1. 理解 DID 的核心思想：为什么需要「差分的差分」
2. 掌握平行趋势假设的检验方法
3. 学会使用 Event Study 设计分析动态效应
4. 理解交错 DID (Staggered DID) 的挑战和解决方案
5. 应用 DID 解决真实业务问题

---

## 业务场景引入

### 场景 1: 电商平台的会员政策

你是一家电商平台的数据科学家。2024年1月1日，公司在**北京地区**推出了新的会员政策（免费包邮），而**上海地区**暂未推出。

你需要回答：**这个会员政策提升了多少用户消费额？**

### 直觉的错误做法

❌ **错误做法 1**：比较北京实施前后的差异
- 问题：可能受季节性、节假日等因素影响

❌ **错误做法 2**：比较北京和上海的差异
- 问题：两个城市本身消费水平就不同

✅ **正确做法**：使用双重差分 (DID)
- 第一次差分：去除时间趋势
- 第二次差分：去除地区差异

---

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# Plotly 配色方案
COLORS = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2994A',
    'info': '#9B51E0',
    'treatment': '#27AE60',
    'control': '#2D9CDB'
}

print("环境准备完成！")

---

## Part 1: DID 的核心直觉

### 1.1 什么是「差分的差分」

想象你在观察两个城市的用户消费变化：

```
         政策前    政策后
北京     100       150      → 增长 50
上海      80       120      → 增长 40
```

**第一次差分（时间维度）**：
- 北京：150 - 100 = 50
- 上海：120 - 80 = 40

**第二次差分（组间差异）**：
- DID = (150 - 100) - (120 - 80) = 50 - 40 = **10**

这个 10 就是政策的因果效应！

### 1.2 DID 的数学表达

$$
\begin{aligned}
\text{DID} &= [E(Y_{\text{treat}, \text{post}}) - E(Y_{\text{treat}, \text{pre}})] \\
           &\quad - [E(Y_{\text{control}, \text{post}}) - E(Y_{\text{control}, \text{pre}})]
\end{aligned}
$$

其中：
- $Y_{\text{treat}, \text{post}}$: 处理组在政策后的结果
- $Y_{\text{treat}, \text{pre}}$: 处理组在政策前的结果
- $Y_{\text{control}, \text{post}}$: 对照组在政策后的结果
- $Y_{\text{control}, \text{pre}}$: 对照组在政策前的结果

---

## 🎓 核心数学推导

### DID 估计量的形式化定义

设 $Y_{it}(1)$ 和 $Y_{it}(0)$ 分别表示个体 $i$ 在时间 $t$ 接受处理和不接受处理的潜在结果。

**处理效应**（ATT, Average Treatment effect on the Treated）：
$$
\tau_{ATT} = E[Y_{1,post}(1) - Y_{1,post}(0) | T=1]
$$

**问题**：$Y_{1,post}(0)$ 不可观测（反事实）。

**平行趋势假设**（Parallel Trends Assumption）：
$$
E[Y_{1,post}(0) - Y_{1,pre}(0) | T=1] = E[Y_{0,post}(0) - Y_{0,pre}(0) | T=0]
$$

**含义**：如果没有政策干预，处理组和对照组的趋势应该是平行的。

**DID 估计量推导**：

基于平行趋势假设，我们可以用对照组的变化来估计处理组的反事实变化：

$$
\begin{aligned}
\tau_{DID} &= [E(Y_{1,post} | T=1) - E(Y_{1,pre} | T=1)] \\
           &\quad - [E(Y_{0,post} | T=0) - E(Y_{0,pre} | T=0)] \\
           &= E[Y_{1,post}(1) - Y_{1,pre}(0) | T=1] \\
           &\quad - E[Y_{0,post}(0) - Y_{0,pre}(0) | T=0] \\
           &= E[Y_{1,post}(1) - Y_{1,post}(0) | T=1] \quad \text{(平行趋势)} \\
           &= \tau_{ATT}
\end{aligned}
$$

### DID 回归模型

DID 可以用回归方程表示：

$$
Y_{it} = \beta_0 + \beta_1 \text{Treat}_i + \beta_2 \text{Post}_t + \beta_3 (\text{Treat}_i \times \text{Post}_t) + \epsilon_{it}
$$

**参数解释**：
- $\beta_0$: 对照组在政策前的平均值
- $\beta_1$: 政策前处理组和对照组的差异（组固定效应）
- $\beta_2$: 对照组在政策后的变化（时间固定效应）
- $\beta_3$: **DID 估计量**（我们关心的因果效应）

**证明** $\beta_3 = \tau_{DID}$：

各组期望值：
- $E[Y | T=0, \text{Post}=0] = \beta_0$
- $E[Y | T=1, \text{Post}=0] = \beta_0 + \beta_1$
- $E[Y | T=0, \text{Post}=1] = \beta_0 + \beta_2$
- $E[Y | T=1, \text{Post}=1] = \beta_0 + \beta_1 + \beta_2 + \beta_3$

计算 DID：
$$
\begin{aligned}
\tau_{DID} &= [(β_0 + β_1 + β_2 + β_3) - (β_0 + β_1)] - [(β_0 + β_2) - β_0] \\
           &= (β_2 + β_3) - β_2 \\
           &= β_3
\end{aligned}
$$

### 带宽选择（IK optimal bandwidth）

对于局部线性回归的最优带宽（用于连续处理的 DID）：

$$
h_{opt} = \left(\frac{C \cdot \sigma^2}{n \cdot m_2^2}\right)^{1/5}
$$

其中：
- $C$: 常数（取决于核函数）
- $\sigma^2$: 条件方差
- $m_2$: 二阶导数
- $n$: 样本量

---

### 1.3 DID 背后的直觉

🎯 **核心思想**：
- **对照组**告诉我们「没有政策时会发生什么」
- **处理组的额外增长**就是政策效应

📊 **生动比喻**：
- 你和朋友一起减肥
- 朋友吃了减肥药，你没吃
- 一个月后，朋友减了 5kg，你减了 2kg
- 减肥药的效果 = 5kg - 2kg = **3kg**
- 为什么不是 5kg？因为你们都在运动、控制饮食（共同趋势）

---

In [ ]:
# 可视化 DID 的直觉
def visualize_did_intuition():
    """
    可视化 DID 的核心直觉
    """
    # 创建示例数据
    data = pd.DataFrame({
        'period': ['政策前', '政策后', '政策前', '政策后'],
        'group': ['北京', '北京', '上海', '上海'],
        'value': [100, 150, 80, 120]
    })
    
    # 创建图表
    fig = go.Figure()
    
    # 北京（处理组）
    beijing = data[data['group'] == '北京']
    fig.add_trace(go.Scatter(
        x=beijing['period'],
        y=beijing['value'],
        mode='lines+markers',
        name='北京（处理组）',
        line=dict(color=COLORS['treatment'], width=3),
        marker=dict(size=12)
    ))
    
    # 上海（对照组）
    shanghai = data[data['group'] == '上海']
    fig.add_trace(go.Scatter(
        x=shanghai['period'],
        y=shanghai['value'],
        mode='lines+markers',
        name='上海（对照组）',
        line=dict(color=COLORS['control'], width=3),
        marker=dict(size=12)
    ))
    
    # 反事实趋势（北京如果没有政策）
    fig.add_trace(go.Scatter(
        x=['政策前', '政策后'],
        y=[100, 140],  # 100 + (120-80) = 140
        mode='lines',
        name='反事实（北京无政策）',
        line=dict(color=COLORS['treatment'], width=2, dash='dash'),
        opacity=0.5
    ))
    
    # 添加箭头标注 DID 效应
    fig.add_annotation(
        x='政策后',
        y=145,
        ax='政策后',
        ay=140,
        xref='x',
        yref='y',
        axref='x',
        ayref='y',
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor=COLORS['danger']
    )
    
    fig.add_annotation(
        x='政策后',
        y=145,
        text='<b>DID 效应 = 10</b>',
        showarrow=False,
        xshift=60,
        font=dict(size=14, color=COLORS['danger'])
    )
    
    # 布局
    fig.update_layout(
        title='双重差分 (DID) 的核心直觉',
        xaxis_title='时间',
        yaxis_title='平均消费额（元）',
        template='plotly_white',
        height=500,
        hovermode='x unified'
    )
    
    return fig

fig = visualize_did_intuition()
fig.show()

print("""
📊 图表解读：
1. 绿线（北京）：政策后增长了 50
2. 蓝线（上海）：同期增长了 40（自然趋势）
3. 虚线：如果北京没有政策，应该也增长 40
4. 红色箭头：额外增长的 10 就是政策效应
""")

---

## Part 2: 基本 DID 设计（两期两组）

### 2.1 回归框架

DID 可以用回归方程表示：

$$
Y_{it} = \beta_0 + \beta_1 \text{Treat}_i + \beta_2 \text{Post}_t + \beta_3 (\text{Treat}_i \times \text{Post}_t) + \epsilon_{it}
$$

**符号说明**：
- $Y_{it}$: 个体 $i$ 在时间 $t$ 的结果变量
- $\text{Treat}_i$: 个体 $i$ 是否在处理组（1=是，0=否）
- $\text{Post}_t$: 时间 $t$ 是否在政策后（1=是，0=否）
- $\text{Treat}_i \times \text{Post}_t$: 交互项
- $\beta_3$: **DID 估计量**（我们关心的因果效应）

### 2.2 参数解释

| 参数 | 含义 | 解释 |
|------|------|------|
| $\beta_0$ | 截距 | 对照组在政策前的平均值 |
| $\beta_1$ | 处理组效应 | 政策前处理组和对照组的差异 |
| $\beta_2$ | 时间效应 | 对照组在政策后的变化 |
| $\beta_3$ | **DID 效应** | 政策的因果效应 |

### 2.3 标准误的选择

⚠️ **重要**：在 DID 设计中，必须使用**聚类标准误** (Clustered Standard Errors)

**为什么？**
- 同一个体的观测值在时间上相关
- 标准 OLS 标准误会低估真实的不确定性
- 聚类在个体层面（或地区层面）

---

In [ ]:
# 生成模拟数据
def generate_did_data(n_treated=500, n_control=500, treatment_effect=10, seed=42):
    """
    生成 DID 模拟数据
    
    参数:
    - n_treated: 处理组样本量
    - n_control: 对照组样本量
    - treatment_effect: 真实处理效应
    - seed: 随机种子
    """
    np.random.seed(seed)
    
    data_list = []
    
    # 处理组（北京）
    for i in range(n_treated):
        user_id = f"beijing_{i}"
        baseline = np.random.normal(100, 15)  # 基线消费
        
        # 政策前
        data_list.append({
            'user_id': user_id,
            'city': 'beijing',
            'period': 'pre',
            'treat': 1,
            'post': 0,
            'spending': baseline + np.random.normal(0, 5)
        })
        
        # 政策后
        time_trend = 40  # 共同时间趋势
        data_list.append({
            'user_id': user_id,
            'city': 'beijing',
            'period': 'post',
            'treat': 1,
            'post': 1,
            'spending': baseline + time_trend + treatment_effect + np.random.normal(0, 5)
        })
    
    # 对照组（上海）
    for i in range(n_control):
        user_id = f"shanghai_{i}"
        baseline = np.random.normal(80, 15)  # 上海基线消费较低
        
        # 政策前
        data_list.append({
            'user_id': user_id,
            'city': 'shanghai',
            'period': 'pre',
            'treat': 0,
            'post': 0,
            'spending': baseline + np.random.normal(0, 5)
        })
        
        # 政策后
        time_trend = 40  # 相同的时间趋势
        data_list.append({
            'user_id': user_id,
            'city': 'shanghai',
            'period': 'post',
            'treat': 0,
            'post': 1,
            'spending': baseline + time_trend + np.random.normal(0, 5)
        })
    
    df = pd.DataFrame(data_list)
    df['treat_post'] = df['treat'] * df['post']  # 交互项
    
    return df

# 生成数据
df = generate_did_data(treatment_effect=10)

print("数据预览：")
print(df.head(10))
print(f"\n数据形状: {df.shape}")
print(f"\n分组统计:")
print(df.groupby(['city', 'period'])['spending'].mean())

In [ ]:
# 手动计算 DID
def manual_did(df):
    """
    手动计算 DID 估计量
    """
    # 计算四个均值
    treat_post = df[(df['treat']==1) & (df['post']==1)]['spending'].mean()
    treat_pre = df[(df['treat']==1) & (df['post']==0)]['spending'].mean()
    control_post = df[(df['treat']==0) & (df['post']==1)]['spending'].mean()
    control_pre = df[(df['treat']==0) & (df['post']==0)]['spending'].mean()
    
    # 第一次差分（时间）
    diff_treat = treat_post - treat_pre
    diff_control = control_post - control_pre
    
    # 第二次差分（组间）
    did = diff_treat - diff_control
    
    print("=" * 60)
    print("手动计算 DID")
    print("=" * 60)
    print(f"\n处理组（北京）：")
    print(f"  政策前: {treat_pre:.2f}")
    print(f"  政策后: {treat_post:.2f}")
    print(f"  差分:   {diff_treat:.2f}")
    
    print(f"\n对照组（上海）：")
    print(f"  政策前: {control_pre:.2f}")
    print(f"  政策后: {control_post:.2f}")
    print(f"  差分:   {diff_control:.2f}")
    
    print(f"\n📊 DID 估计量: {did:.2f}")
    print(f"（真实效应: 10.00）")
    print("=" * 60)
    
    return did

manual_did_estimate = manual_did(df)

In [ ]:
# 回归估计 DID
def regression_did(df):
    """
    使用回归估计 DID
    """
    # OLS 回归
    model = smf.ols('spending ~ treat + post + treat_post', data=df).fit()
    
    # 聚类标准误（按用户聚类）
    model_clustered = smf.ols('spending ~ treat + post + treat_post', data=df).fit(
        cov_type='cluster',
        cov_kwds={'groups': df['user_id']}
    )
    
    print("=" * 60)
    print("回归估计 DID")
    print("=" * 60)
    print("\n回归方程: spending ~ treat + post + treat_post")
    print("\n标准 OLS 标准误:")
    print(model.summary().tables[1])
    
    print("\n聚类标准误（按用户聚类）:")
    print(model_clustered.summary().tables[1])
    
    print(f"\n📊 DID 估计量 (treat_post): {model.params['treat_post']:.2f}")
    print(f"   标准误 (OLS): {model.bse['treat_post']:.4f}")
    print(f"   标准误 (聚类): {model_clustered.bse['treat_post']:.4f}")
    print(f"   p-value: {model_clustered.pvalues['treat_post']:.4f}")
    print("=" * 60)
    
    return model_clustered

model = regression_did(df)

### 💡 关键发现

1. **手动计算和回归估计一致**：两种方法得到相同的 DID 估计量
2. **聚类标准误更大**：考虑了个体内相关性后，不确定性增加
3. **回归框架的优势**：可以方便地加入控制变量、计算置信区间

---

## Part 3: 平行趋势假设 (Parallel Trends Assumption)

### 3.1 假设的含义

DID 的核心假设：**如果没有政策干预，处理组和对照组的趋势应该是平行的**。

$$
E[Y_{1t}(0) - Y_{1,t-1}(0)] = E[Y_{0t}(0) - Y_{0,t-1}(0)]
$$

**通俗解释**：
- 如果北京没有推出会员政策
- 北京和上海的消费增长应该是一样的
- 这是一个**反事实假设**，无法直接验证

### 3.2 为什么平行趋势重要？

如果违反平行趋势假设，DID 估计是**有偏**的：

- 处理组本身增长更快 → DID 高估政策效应
- 处理组本身增长更慢 → DID 低估政策效应

### 3.3 检验方法

虽然无法直接验证，但我们可以：

1. **图形化检验**：观察政策前的趋势
2. **统计检验**：测试政策前各期的差异
3. **安慰剂检验**：使用假的政策时间点

---

In [ ]:
# 生成多期数据用于平行趋势检验
def generate_multi_period_data(n_treated=500, n_control=500, n_periods=10, 
                               treatment_time=6, treatment_effect=10, seed=42):
    """
    生成多期 DID 数据
    
    参数:
    - n_periods: 总时期数
    - treatment_time: 政策开始时期（从0开始）
    """
    np.random.seed(seed)
    
    data_list = []
    
    # 处理组
    for i in range(n_treated):
        user_id = f"beijing_{i}"
        baseline = np.random.normal(100, 15)
        individual_trend = np.random.normal(0, 2)  # 个体特定趋势
        
        for t in range(n_periods):
            time_effect = 4 * t  # 共同时间趋势
            is_post = 1 if t >= treatment_time else 0
            treatment = treatment_effect if is_post else 0
            
            data_list.append({
                'user_id': user_id,
                'city': 'beijing',
                'period': t,
                'treat': 1,
                'post': is_post,
                'spending': baseline + time_effect + individual_trend * t + treatment + np.random.normal(0, 5)
            })
    
    # 对照组
    for i in range(n_control):
        user_id = f"shanghai_{i}"
        baseline = np.random.normal(80, 15)
        individual_trend = np.random.normal(0, 2)
        
        for t in range(n_periods):
            time_effect = 4 * t
            
            data_list.append({
                'user_id': user_id,
                'city': 'shanghai',
                'period': t,
                'treat': 0,
                'post': 0,  # 对照组始终为0
                'spending': baseline + time_effect + individual_trend * t + np.random.normal(0, 5)
            })
    
    df = pd.DataFrame(data_list)
    return df, treatment_time

# 生成多期数据
df_multi, treatment_time = generate_multi_period_data()

print("多期数据预览：")
print(df_multi.head(20))
print(f"\n政策时间点: 第 {treatment_time} 期")

In [ ]:
# 图形化检验：平行趋势
def plot_parallel_trends(df, treatment_time):
    """
    可视化平行趋势假设
    """
    # 计算每期的平均消费
    trends = df.groupby(['period', 'city'])['spending'].mean().reset_index()
    
    fig = go.Figure()
    
    # 北京（处理组）
    beijing = trends[trends['city'] == 'beijing']
    fig.add_trace(go.Scatter(
        x=beijing['period'],
        y=beijing['spending'],
        mode='lines+markers',
        name='北京（处理组）',
        line=dict(color=COLORS['treatment'], width=3),
        marker=dict(size=8)
    ))
    
    # 上海（对照组）
    shanghai = trends[trends['city'] == 'shanghai']
    fig.add_trace(go.Scatter(
        x=shanghai['period'],
        y=shanghai['spending'],
        mode='lines+markers',
        name='上海（对照组）',
        line=dict(color=COLORS['control'], width=3),
        marker=dict(size=8)
    ))
    
    # 添加政策时间线
    fig.add_vline(
        x=treatment_time - 0.5,
        line_dash="dash",
        line_color=COLORS['danger'],
        line_width=2,
        annotation_text="政策开始",
        annotation_position="top"
    )
    
    # 添加背景色区分政策前后
    fig.add_vrect(
        x0=-0.5, x1=treatment_time-0.5,
        fillcolor="lightgray", opacity=0.1,
        layer="below", line_width=0,
        annotation_text="政策前", annotation_position="top left"
    )
    
    fig.add_vrect(
        x0=treatment_time-0.5, x1=9.5,
        fillcolor="lightgreen", opacity=0.1,
        layer="below", line_width=0,
        annotation_text="政策后", annotation_position="top right"
    )
    
    fig.update_layout(
        title='平行趋势检验：政策前趋势是否平行？',
        xaxis_title='时期',
        yaxis_title='平均消费额（元）',
        template='plotly_white',
        height=500,
        hovermode='x unified'
    )
    
    return fig

fig = plot_parallel_trends(df_multi, treatment_time)
fig.show()

print("""
📊 如何判断平行趋势？
1. 观察政策前（灰色区域）两条线是否平行
2. 如果政策前趋势相似，则支持平行趋势假设
3. 政策后（绿色区域）出现分离，这是政策效应
""")

In [ ]:
# 统计检验：Lead 检验
def lead_test(df, treatment_time):
    """
    Lead 检验：测试政策前各期是否有显著差异
    
    原理：如果平行趋势成立，政策前各期的处理组-对照组差异应该稳定
    """
    # 只使用政策前的数据
    df_pre = df[df['period'] < treatment_time].copy()
    
    # 创建每期的虚拟变量
    for t in df_pre['period'].unique():
        df_pre[f'period_{t}'] = (df_pre['period'] == t).astype(int)
        df_pre[f'treat_period_{t}'] = df_pre['treat'] * df_pre[f'period_{t}']
    
    # 构建回归公式
    period_dummies = ' + '.join([f'period_{t}' for t in range(1, treatment_time)])
    interaction_terms = ' + '.join([f'treat_period_{t}' for t in range(1, treatment_time)])
    formula = f'spending ~ treat + {period_dummies} + {interaction_terms}'
    
    # 回归
    model = smf.ols(formula, data=df_pre).fit(
        cov_type='cluster',
        cov_kwds={'groups': df_pre['user_id']}
    )
    
    # 提取交互项系数
    lead_coeffs = []
    for t in range(1, treatment_time):
        param_name = f'treat_period_{t}'
        if param_name in model.params:
            lead_coeffs.append({
                'period': t,
                'coef': model.params[param_name],
                'se': model.bse[param_name],
                'pvalue': model.pvalues[param_name]
            })
    
    lead_df = pd.DataFrame(lead_coeffs)
    
    print("=" * 60)
    print("Lead 检验结果")
    print("=" * 60)
    print("\n检验原理：如果平行趋势成立，政策前各期的处理效应应该为0\n")
    print(lead_df.to_string(index=False))
    
    significant = lead_df[lead_df['pvalue'] < 0.05]
    if len(significant) > 0:
        print(f"\n⚠️  警告：有 {len(significant)} 个时期的系数显著不为0")
        print("   这可能违反了平行趋势假设")
    else:
        print("\n✅ 所有时期的系数都不显著，支持平行趋势假设")
    
    print("=" * 60)
    
    return lead_df

lead_results = lead_test(df_multi, treatment_time)

### 🎯 TODO 练习 1: 安慰剂检验

**任务**：实现安慰剂检验 (Placebo Test)

**思路**：
1. 使用政策前的数据
2. 假设一个「假的」政策时间点（比如真实政策前2期）
3. 估计 DID，如果显著，说明可能违反平行趋势

**提示**：
```python
# 只使用政策前的数据
df_placebo = df_multi[df_multi['period'] < treatment_time].copy()

# 设置假的政策时间点
fake_treatment_time = treatment_time - 2
df_placebo['fake_post'] = (df_placebo['period'] >= fake_treatment_time).astype(int)
df_placebo['treat_fake_post'] = df_placebo['treat'] * df_placebo['fake_post']

# 回归估计
# TODO: 补全代码
```

In [ ]:
# 实现安慰剂检验
def placebo_test(df, treatment_time):
    """
    安慰剂检验 - 完整实现
    """
    # 只使用政策前的数据
    df_placebo = df[df['period'] < treatment_time].copy()
    
    # 设置假的政策时间点（真实政策前2期）
    fake_treatment_time = treatment_time - 2
    
    # 创建 fake_post 变量
    df_placebo['fake_post'] = (df_placebo['period'] >= fake_treatment_time).astype(int)
    
    # 创建交互项
    df_placebo['treat_fake_post'] = df_placebo['treat'] * df_placebo['fake_post']
    
    # 回归估计
    model = smf.ols('spending ~ treat + fake_post + treat_fake_post', 
                    data=df_placebo).fit(
        cov_type='cluster',
        cov_kwds={'groups': df_placebo['user_id']}
    )
    
    # 提取结果
    placebo_effect = model.params['treat_fake_post']
    placebo_se = model.bse['treat_fake_post']
    placebo_pvalue = model.pvalues['treat_fake_post']
    
    print("=" * 60)
    print("安慰剂检验结果")
    print("=" * 60)
    print(f"\n假的政策时间点: 第 {fake_treatment_time} 期")
    print(f"真实政策时间点: 第 {treatment_time} 期\n")
    
    print(f"安慰剂 DID 估计量: {placebo_effect:.2f}")
    print(f"标准误: {placebo_se:.4f}")
    print(f"p-value: {placebo_pvalue:.4f}")
    
    if placebo_pvalue < 0.05:
        print("\n⚠️  警告：安慰剂效应显著，可能违反平行趋势假设")
        print("   这表明在真实政策之前，两组已经存在差异趋势")
    else:
        print("\n✅ 安慰剂效应不显著，支持平行趋势假设")
        print("   这增强了我们对真实DID估计的信心")
    
    print("=" * 60)
    
    return model

# 运行安慰剂检验
placebo_result = placebo_test(df_multi, treatment_time)

In [ ]:
# 参考答案（先自己尝试，再查看）
def placebo_test_solution(df, treatment_time):
    """
    安慰剂检验 - 参考答案
    """
    df_placebo = df[df['period'] < treatment_time].copy()
    fake_treatment_time = treatment_time - 2
    
    df_placebo['fake_post'] = (df_placebo['period'] >= fake_treatment_time).astype(int)
    df_placebo['treat_fake_post'] = df_placebo['treat'] * df_placebo['fake_post']
    
    model = smf.ols('spending ~ treat + fake_post + treat_fake_post', 
                    data=df_placebo).fit(
        cov_type='cluster',
        cov_kwds={'groups': df_placebo['user_id']}
    )
    
    print("=" * 60)
    print("安慰剂检验结果")
    print("=" * 60)
    print(f"\n假的政策时间点: 第 {fake_treatment_time} 期")
    print(f"真实政策时间点: 第 {treatment_time} 期\n")
    
    print(model.summary().tables[1])
    
    placebo_effect = model.params['treat_fake_post']
    placebo_pvalue = model.pvalues['treat_fake_post']
    
    print(f"\n📊 安慰剂 DID 估计量: {placebo_effect:.2f}")
    print(f"   p-value: {placebo_pvalue:.4f}")
    
    if placebo_pvalue < 0.05:
        print("\n⚠️  警告：安慰剂效应显著，可能违反平行趋势假设")
    else:
        print("\n✅ 安慰剂效应不显著，支持平行趋势假设")
    
    print("=" * 60)
    
    return model

placebo_test_solution(df_multi, treatment_time)

---

## Part 4: Event Study 设计

### 4.1 什么是 Event Study？

Event Study 是 DID 的扩展，允许我们：
1. **可视化平行趋势**：观察政策前的趋势
2. **估计动态效应**：政策效应如何随时间变化
3. **检验滞后效应**：政策是否有延迟影响

### 4.2 回归方程

$$
Y_{it} = \alpha_i + \lambda_t + \sum_{k \neq -1} \beta_k \cdot \mathbb{1}[t - t^* = k] \cdot \text{Treat}_i + \epsilon_{it}
$$

**符号说明**：
- $\alpha_i$: 个体固定效应
- $\lambda_t$: 时间固定效应
- $t^*$: 政策时间点
- $k$: 相对时间（与政策时间的距离）
- $\beta_k$: 相对时间 $k$ 的处理效应
- $k = -1$ 作为基准期（归一化为0）

### 4.3 Event Study 图的解读

- **政策前** ($k < 0$)：系数应该接近0（平行趋势）
- **政策时** ($k = 0$)：开始出现效应
- **政策后** ($k > 0$)：观察效应的演变

---

In [ ]:
# Event Study 估计
def event_study(df, treatment_time):
    """
    Event Study 估计
    """
    df = df.copy()
    
    # 计算相对时间
    df['rel_time'] = df['period'] - treatment_time
    
    # 创建相对时间虚拟变量（排除 -1 作为基准）
    rel_times = sorted(df['rel_time'].unique())
    rel_times = [t for t in rel_times if t != -1]  # 排除基准期
    
    for t in rel_times:
        df[f'rel_time_{t}'] = (df['rel_time'] == t).astype(int)
        df[f'treat_rel_time_{t}'] = df['treat'] * df[f'rel_time_{t}']
    
    # 构建回归公式
    period_dummies = ' + '.join([f'C(period)'])
    interaction_terms = ' + '.join([f'treat_rel_time_{t}' for t in rel_times])
    formula = f'spending ~ treat + {period_dummies} + {interaction_terms}'
    
    # 回归
    model = smf.ols(formula, data=df).fit(
        cov_type='cluster',
        cov_kwds={'groups': df['user_id']}
    )
    
    # 提取系数
    coeffs = []
    for t in rel_times:
        param_name = f'treat_rel_time_{t}'
        if param_name in model.params:
            coeffs.append({
                'rel_time': t,
                'coef': model.params[param_name],
                'se': model.bse[param_name],
                'ci_lower': model.params[param_name] - 1.96 * model.bse[param_name],
                'ci_upper': model.params[param_name] + 1.96 * model.bse[param_name]
            })
    
    # 添加基准期（系数为0）
    coeffs.append({
        'rel_time': -1,
        'coef': 0,
        'se': 0,
        'ci_lower': 0,
        'ci_upper': 0
    })
    
    event_df = pd.DataFrame(coeffs).sort_values('rel_time')
    
    return event_df, model

event_df, event_model = event_study(df_multi, treatment_time)

print("Event Study 估计结果：")
print(event_df.to_string(index=False))

In [ ]:
# 绘制 Event Study 图
def plot_event_study(event_df):
    """
    绘制 Event Study 图
    """
    fig = go.Figure()
    
    # 点估计
    fig.add_trace(go.Scatter(
        x=event_df['rel_time'],
        y=event_df['coef'],
        mode='markers+lines',
        name='DID 估计量',
        marker=dict(size=10, color=COLORS['primary']),
        line=dict(color=COLORS['primary'], width=2)
    ))
    
    # 95% 置信区间
    fig.add_trace(go.Scatter(
        x=list(event_df['rel_time']) + list(event_df['rel_time'][::-1]),
        y=list(event_df['ci_upper']) + list(event_df['ci_lower'][::-1]),
        fill='toself',
        fillcolor='rgba(45, 156, 219, 0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        name='95% 置信区间',
        showlegend=True
    ))
    
    # 零线
    fig.add_hline(
        y=0,
        line_dash="dash",
        line_color="gray",
        line_width=1
    )
    
    # 政策时间线
    fig.add_vline(
        x=-0.5,
        line_dash="dash",
        line_color=COLORS['danger'],
        line_width=2,
        annotation_text="政策开始",
        annotation_position="top"
    )
    
    fig.update_layout(
        title='Event Study 图：政策效应的动态演变',
        xaxis_title='相对时间（相对于政策时间）',
        yaxis_title='处理效应（元）',
        template='plotly_white',
        height=500,
        hovermode='x unified'
    )
    
    return fig

fig = plot_event_study(event_df)
fig.show()

print("""
📊 Event Study 图解读：
1. 政策前（负值）：系数应该接近0且不显著（支持平行趋势）
2. 政策时（0）：开始出现正向效应
3. 政策后（正值）：观察效应是否持续、增强或减弱
4. 置信区间：如果包含0，说明该期效应不显著
""")

---

## Part 5: 交错 DID (Staggered DID)

### 5.1 什么是交错 DID？

在现实中，政策往往不是同时在所有地区实施：

```
         2024-01  2024-02  2024-03  2024-04
北京       ✅       ✅       ✅       ✅
上海       ⭕       ⭕       ✅       ✅
深圳       ⭕       ⭕       ⭕       ✅
广州       ⭕       ⭕       ⭕       ⭕
```

这种**异质性处理时间** (Staggered Treatment Timing) 的设计称为交错 DID。

### 5.2 TWFE 的问题

传统的双向固定效应 (Two-Way Fixed Effects, TWFE) 估计量：

$$
Y_{it} = \alpha_i + \lambda_t + \beta \cdot \text{Treat}_{it} + \epsilon_{it}
$$

在交错 DID 中存在**严重问题**：

1. **负权重问题**：某些处理效应被赋予负权重
2. **异质性偏差**：如果处理效应随时间变化，TWFE 有偏
3. **禁忌比较**：已处理组被用作未处理组的对照

### 5.3 现代解决方案

**Callaway & Sant'Anna (2021)**：
- 避免禁忌比较
- 允许异质性处理效应
- 提供多种聚合方式

**Sun & Abraham (2021)**：
- 交互加权估计量
- 兼容 Event Study 框架

**De Chaisemartin & D'Haultfoeuille (2020)**：
- DID_M 估计量
- 诊断工具

### 5.4 简化示例

由于完整的交错 DID 实现较复杂（需要专门的包如 `did` 或 `csdid`），这里我们展示核心思想：

---

In [ ]:
# 生成交错 DID 数据
def generate_staggered_did_data(n_cities=4, n_users_per_city=200, n_periods=12, seed=42):
    """
    生成交错 DID 数据
    """
    np.random.seed(seed)
    
    cities = ['Beijing', 'Shanghai', 'Shenzhen', 'Guangzhou']
    treatment_times = [3, 6, 9, None]  # None 表示始终不处理
    
    data_list = []
    
    for city, treat_time in zip(cities, treatment_times):
        for i in range(n_users_per_city):
            user_id = f"{city}_{i}"
            baseline = np.random.normal(100, 15)
            
            for t in range(n_periods):
                time_effect = 3 * t
                
                # 是否已处理
                is_treated = 0
                if treat_time is not None and t >= treat_time:
                    is_treated = 1
                
                # 处理效应（随时间衰减）
                if is_treated:
                    time_since_treatment = t - treat_time
                    treatment_effect = 15 * np.exp(-0.1 * time_since_treatment)
                else:
                    treatment_effect = 0
                
                data_list.append({
                    'user_id': user_id,
                    'city': city,
                    'period': t,
                    'treat_time': treat_time if treat_time is not None else 999,
                    'is_treated': is_treated,
                    'spending': baseline + time_effect + treatment_effect + np.random.normal(0, 5)
                })
    
    df = pd.DataFrame(data_list)
    return df

# 生成交错 DID 数据
df_staggered = generate_staggered_did_data()

print("交错 DID 数据预览：")
print(df_staggered.head(20))
print("\n各城市的处理时间：")
print(df_staggered.groupby('city')['treat_time'].first())

In [ ]:
# 可视化交错 DID
def plot_staggered_did(df):
    """
    可视化交错 DID 的处理模式
    """
    # 计算每个城市每期的平均消费
    trends = df.groupby(['period', 'city'])['spending'].mean().reset_index()
    
    fig = go.Figure()
    
    colors_map = {
        'Beijing': COLORS['success'],
        'Shanghai': COLORS['primary'],
        'Shenzhen': COLORS['warning'],
        'Guangzhou': COLORS['danger']
    }
    
    for city in trends['city'].unique():
        city_data = trends[trends['city'] == city]
        fig.add_trace(go.Scatter(
            x=city_data['period'],
            y=city_data['spending'],
            mode='lines+markers',
            name=city,
            line=dict(color=colors_map[city], width=2),
            marker=dict(size=6)
        ))
    
    # 添加处理时间线
    treatment_times = df.groupby('city')['treat_time'].first()
    for city, treat_time in treatment_times.items():
        if treat_time < 999:
            fig.add_vline(
                x=treat_time - 0.5,
                line_dash="dot",
                line_color=colors_map[city],
                line_width=1,
                annotation_text=f"{city}",
                annotation_position="top"
            )
    
    fig.update_layout(
        title='交错 DID：不同城市在不同时间接受处理',
        xaxis_title='时期',
        yaxis_title='平均消费额（元）',
        template='plotly_white',
        height=500,
        hovermode='x unified'
    )
    
    return fig

fig = plot_staggered_did(df_staggered)
fig.show()

print("""
📊 交错 DID 的特点：
1. 不同城市在不同时间接受处理（虚线标记）
2. 处理后的轨迹出现跳跃
3. 广州始终未处理，可作为对照组
4. 需要小心处理「已处理」vs「未处理」的比较
""")

In [ ]:
# TWFE 估计（可能有偏）
def twfe_estimation(df):
    """
    传统的双向固定效应估计
    """
    model = smf.ols('spending ~ C(city) + C(period) + is_treated', data=df).fit(
        cov_type='cluster',
        cov_kwds={'groups': df['user_id']}
    )
    
    print("=" * 60)
    print("TWFE 估计结果")
    print("=" * 60)
    print("\n回归方程: spending ~ city_fe + period_fe + is_treated\n")
    print(model.summary().tables[1])
    
    twfe_effect = model.params['is_treated']
    print(f"\n📊 TWFE 估计的处理效应: {twfe_effect:.2f}")
    print("\n⚠️  注意：在交错 DID 设计中，TWFE 估计量可能有偏")
    print("   原因：")
    print("   1. 异质性处理效应（效应随时间衰减）")
    print("   2. 禁忌比较（已处理组作为对照）")
    print("   3. 负权重问题")
    print("=" * 60)
    
    return model

twfe_model = twfe_estimation(df_staggered)

### 5.5 现代估计方法的简化实现

完整的 Callaway-Sant'Anna 或 Sun-Abraham 估计量需要专门的包。这里我们展示核心思想：

**策略**：
1. 将样本分组（按处理时间）
2. 对每个组单独估计 DID
3. 加权平均得到总体效应

---

In [ ]:
# 简化的 Callaway-Sant'Anna 思想
def simplified_callaway_santanna(df):
    """
    简化版的 Callaway-Sant'Anna 估计
    
    核心思想：
    1. 对每个处理组（cohort），使用未处理组作为对照
    2. 估计每个 cohort 的 ATT
    3. 加权平均
    """
    results = []
    
    # 获取所有处理时间（排除始终未处理的）
    treat_times = df[df['treat_time'] < 999]['treat_time'].unique()
    
    for treat_time in sorted(treat_times):
        # 该 cohort（在 treat_time 接受处理的城市）
        cohort = df[df['treat_time'] == treat_time].copy()
        
        # 对照组（始终未处理的城市）
        control = df[df['treat_time'] == 999].copy()
        
        # 合并
        did_data = pd.concat([cohort, control])
        
        # 创建 post 变量
        did_data['post'] = (did_data['period'] >= treat_time).astype(int)
        did_data['treat'] = (did_data['treat_time'] == treat_time).astype(int)
        did_data['treat_post'] = did_data['treat'] * did_data['post']
        
        # DID 估计
        model = smf.ols('spending ~ treat + post + treat_post', data=did_data).fit(
            cov_type='cluster',
            cov_kwds={'groups': did_data['user_id']}
        )
        
        results.append({
            'treat_time': treat_time,
            'att': model.params['treat_post'],
            'se': model.bse['treat_post'],
            'n': len(cohort) // 12  # 用户数
        })
    
    results_df = pd.DataFrame(results)
    
    # 加权平均（按样本量）
    total_n = results_df['n'].sum()
    overall_att = (results_df['att'] * results_df['n']).sum() / total_n
    
    print("=" * 60)
    print("简化版 Callaway-Sant'Anna 估计")
    print("=" * 60)
    print("\n各 Cohort 的 ATT：\n")
    print(results_df.to_string(index=False))
    
    print(f"\n📊 加权平均 ATT: {overall_att:.2f}")
    print("\n💡 这个估计避免了 TWFE 的问题：")
    print("   1. 只使用未处理组作为对照")
    print("   2. 允许每个 cohort 有不同的处理效应")
    print("=" * 60)
    
    return results_df, overall_att

cs_results, cs_att = simplified_callaway_santanna(df_staggered)

---

## Part 6: 业务案例

### 案例 1: 最低工资政策效果

**背景**：
- 1992年，新泽西州提高最低工资，宾夕法尼亚州未提高
- 研究问题：提高最低工资是否会减少就业？

**数据**：
- 快餐店的就业人数
- 政策前后各一期

**经典论文**：Card & Krueger (1994), *Minimum Wages and Employment*

---

In [ ]:
# 案例 1: 最低工资政策（模拟数据）
def minimum_wage_case_study():
    """
    模拟最低工资政策的 DID 分析
    """
    np.random.seed(42)
    
    data_list = []
    
    # 新泽西州（处理组）- 400家快餐店
    for i in range(400):
        store_id = f"NJ_{i}"
        baseline_emp = np.random.normal(20, 5)  # 基线就业人数
        
        # 政策前
        data_list.append({
            'store_id': store_id,
            'state': 'New Jersey',
            'period': 'Before',
            'treat': 1,
            'post': 0,
            'employment': baseline_emp + np.random.normal(0, 2)
        })
        
        # 政策后（最低工资提高后，就业反而增加了！）
        data_list.append({
            'store_id': store_id,
            'state': 'New Jersey',
            'period': 'After',
            'treat': 1,
            'post': 1,
            'employment': baseline_emp + 0.5 + 2.5 + np.random.normal(0, 2)  # 增加2.5人
        })
    
    # 宾夕法尼亚州（对照组）- 300家快餐店
    for i in range(300):
        store_id = f"PA_{i}"
        baseline_emp = np.random.normal(20, 5)
        
        # 政策前
        data_list.append({
            'store_id': store_id,
            'state': 'Pennsylvania',
            'period': 'Before',
            'treat': 0,
            'post': 0,
            'employment': baseline_emp + np.random.normal(0, 2)
        })
        
        # 政策后（自然增长）
        data_list.append({
            'store_id': store_id,
            'state': 'Pennsylvania',
            'period': 'After',
            'treat': 0,
            'post': 1,
            'employment': baseline_emp + 0.5 + np.random.normal(0, 2)  # 增加0.5人
        })
    
    df = pd.DataFrame(data_list)
    df['treat_post'] = df['treat'] * df['post']
    
    # DID 分析
    print("=" * 60)
    print("案例 1: 最低工资政策对就业的影响")
    print("=" * 60)
    print("\n研究背景：")
    print("  1992年，新泽西州将最低工资从 $4.25 提高到 $5.05")
    print("  宾夕法尼亚州维持 $4.25 不变")
    print("  研究问题：提高最低工资是否会减少就业？")
    
    # 描述性统计
    print("\n" + "=" * 60)
    print("描述性统计")
    print("=" * 60)
    stats = df.groupby(['state', 'period'])['employment'].agg(['mean', 'std', 'count'])
    print(stats)
    
    # 回归估计
    model = smf.ols('employment ~ treat + post + treat_post', data=df).fit(
        cov_type='cluster',
        cov_kwds={'groups': df['store_id']}
    )
    
    print("\n" + "=" * 60)
    print("DID 回归结果")
    print("=" * 60)
    print(model.summary().tables[1])
    
    did_effect = model.params['treat_post']
    pvalue = model.pvalues['treat_post']
    
    print(f"\n📊 DID 估计量: {did_effect:.2f} 人")
    print(f"   p-value: {pvalue:.4f}")
    
    if pvalue < 0.05:
        if did_effect > 0:
            print("\n✅ 结论：提高最低工资显著增加了就业！")
        else:
            print("\n⚠️  结论：提高最低工资显著减少了就业")
    else:
        print("\n结论：提高最低工资对就业没有显著影响")
    
    print("\n💡 经济学启示：")
    print("   传统理论认为提高最低工资会减少就业（需求曲线）")
    print("   但实证研究发现效应可能为正或不显著")
    print("   可能原因：单一雇主模型、效率工资、工人流动率降低等")
    print("=" * 60)
    
    return df, model

df_minwage, model_minwage = minimum_wage_case_study()

### 案例 2: 平台政策变更评估

**背景**：
- 外卖平台在部分城市推出「无接触配送」功能
- 评估该功能对订单量的影响

---

### 🎯 TODO 练习 2: 平台政策变更案例

**任务**：分析「无接触配送」功能对订单量的影响

**数据生成**：
```python
# 处理组：北京、上海（2024-03-01 推出）
# 对照组：深圳、广州（未推出）
# 时间：2024-01 到 2024-06（共6个月）
# 真实效应：订单量增加 15%
```

**要求**：
1. 生成模拟数据
2. 进行 DID 估计
3. 绘制趋势图
4. 检验平行趋势假设

**提示**：参考前面的案例代码

In [ ]:
# 平台政策变更案例 - 完整实现
def contactless_delivery_case_study():
    """
    无接触配送功能的 DID 分析 - 参考答案
    """
    np.random.seed(42)
    
    # 时间跨度：2022-01 到 2024-06（共30个月）
    months = pd.date_range('2022-01', '2024-07', freq='M')
    n_months = len(months)
    treatment_month_idx = 14  # 2023年3月（索引从0开始）
    
    # 城市列表
    cities = {
        '北京': {'treated': True, 'baseline': 1000},
        '上海': {'treated': True, 'baseline': 950},
        '深圳': {'treated': False, 'baseline': 800},
        '广州': {'treated': False, 'baseline': 820}
    }
    
    data_list = []
    
    for city_name, city_info in cities.items():
        baseline = city_info['baseline']
        is_treated = city_info['treated']
        
        for t in range(n_months):
            # 共同时间趋势：每月增长5%
            time_effect = baseline * 0.05 * t / 12
            
            # 季节性效应
            seasonal = 50 * np.sin(2 * np.pi * t / 12)
            
            # 处理效应：无接触配送提升15%
            is_post = t >= treatment_month_idx
            treatment_effect = baseline * 0.15 if (is_treated and is_post) else 0
            
            # 噪声
            noise = np.random.normal(0, 30)
            
            # 订单量
            orders = baseline + time_effect + seasonal + treatment_effect + noise
            
            data_list.append({
                'month': months[t],
                'city': city_name,
                'orders': orders,
                'treated': 1 if is_treated else 0,
                'post': 1 if is_post else 0
            })
    
    df = pd.DataFrame(data_list)
    df['treat_post'] = df['treated'] * df['post']
    
    print("=" * 70)
    print("案例 2: 无接触配送功能的 DID 分析")
    print("=" * 70)
    
    # 1. 可视化趋势
    fig = go.Figure()
    
    for city in cities.keys():
        city_data = df[df['city'] == city]
        color = COLORS['treatment'] if cities[city]['treated'] else COLORS['control']
        name = f"{city}（{'处理组' if cities[city]['treated'] else '对照组'}）"
        
        fig.add_trace(go.Scatter(
            x=city_data['month'],
            y=city_data['orders'],
            name=name,
            line=dict(color=color, width=2),
            mode='lines+markers'
        ))
    
    fig.add_vline(
        x=months[treatment_month_idx],
        line_dash="dash",
        line_color=COLORS['danger'],
        annotation_text="功能上线"
    )
    
    fig.update_layout(
        title='各城市订单量趋势',
        xaxis_title='月份',
        yaxis_title='日均订单量',
        template='plotly_white',
        height=500,
        hovermode='x unified'
    )
    
    fig.show()
    
    # 2. DID 估计
    model = smf.ols('orders ~ treated + post + treat_post', data=df).fit(
        cov_type='cluster',
        cov_kwds={'groups': df['city']}
    )
    
    print("\n[DID 回归结果]")
    print(model.summary().tables[1])
    
    did_effect = model.params['treat_post']
    did_se = model.bse['treat_post']
    did_pvalue = model.pvalues['treat_post']
    
    print(f"\n📊 DID 估计量: {did_effect:.2f} 单/天")
    print(f"   标准误: {did_se:.2f}")
    print(f"   p-value: {did_pvalue:.4f}")
    
    # 3. 平行趋势检验
    print("\n[平行趋势检验]")
    df_pre = df[df['post'] == 0].copy()
    df_pre['month_num'] = (df_pre['month'] - df_pre['month'].min()).dt.days / 30
    df_pre['treat_month'] = df_pre['treated'] * df_pre['month_num']
    
    trend_model = smf.ols('orders ~ treated + month_num + treat_month', data=df_pre).fit()
    trend_coef = trend_model.params['treat_month']
    trend_pvalue = trend_model.pvalues['treat_month']
    
    print(f"   处理组与对照组趋势差异: {trend_coef:.4f}")
    print(f"   p-value: {trend_pvalue:.4f}")
    
    if trend_pvalue > 0.05:
        print("   ✅ 政策前趋势无显著差异，支持平行趋势假设")
    else:
        print("   ⚠️  政策前趋势存在差异，可能违反平行趋势假设")
    
    # 4. 业务建议
    print("\n[业务建议]")
    baseline_orders = df[(df['treated'] == 1) & (df['post'] == 0)]['orders'].mean()
    pct_increase = (did_effect / baseline_orders) * 100
    
    print(f"  1. 无接触配送使订单量提升 {did_effect:.0f} 单/天 ({pct_increase:.1f}%)")
    print(f"  2. 建议在深圳、广州尽快上线该功能")
    print(f"  3. 预计可带来月增量: {did_effect * 30:.0f} 单")
    
    print("=" * 70)
    
    return df, model

# 运行案例分析
df_case2, model_case2 = contactless_delivery_case_study()

---

## Part 7: 面试高频题模拟

### 🎯 面试题 1：DID 的核心假设是什么？如何验证？

**标准答案**：

DID 的核心假设是 **平行趋势假设** (Parallel Trends Assumption)：

**形式化定义**：
$$
E[Y_{it}(0) | T=1] - E[Y_{i,t-1}(0) | T=1] = E[Y_{it}(0) | T=0] - E[Y_{i,t-1}(0) | T=0]
$$

**通俗解释**：如果没有政策干预，处理组和对照组的趋势应该是平行的。

**验证方法**：

1. **图形化检验**：
   - 绘制政策前的趋势图
   - 观察两组是否平行

2. **统计检验 - Lead 检验**：
   - 在政策前各期添加虚拟交互项
   - 检验系数是否显著为0
   - 公式：$Y_{it} = \alpha + \sum_{k=-K}^{-2} \beta_k D_i \cdot \mathbb{1}[t=k] + \gamma_t + \delta_i + \epsilon_{it}$

3. **安慰剂检验 (Placebo Test)**：
   - 在政策前设置假的政策时间点
   - 估计"假DID"，应该不显著

4. **Event Study**：
   - 估计每期的处理效应
   - 政策前各期的系数应该接近0

**面试加分回答**：
- "平行趋势是不可检验的反事实假设，我们只能检验政策前的趋势"
- "即使政策前平行，也不能保证政策后继续平行"
- "可以通过多种方法相互验证，增强可信度"

---

### 🎯 面试题 2：合成控制法适用于什么场景？

**标准答案**：

合成控制法适用于以下场景：

1. **处理单位数量少**：
   - 通常只有1个处理单位
   - 例如：某个城市、某个国家、某个公司

2. **找不到完美的对照组**：
   - 没有一个单位与处理单位完全相似
   - 需要用多个单位的加权组合来构建"虚拟"对照

3. **处理是集中式事件**：
   - 某个法案通过
   - 某个政策实施
   - 某个外部冲击

4. **有多期观测数据**：
   - 需要足够的前处理期数据来估计权重
   - 通常至少需要10期以上

**与DID的对比**：

| 维度 | DID | 合成控制 |
|------|-----|----------|
| 处理单位数 | 多个 | 通常1个 |
| 对照组构建 | 简单平均 | 优化权重 |
| 关键假设 | 平行趋势 | 可合成性 |
| 推断方法 | 标准误 | Placebo Tests |

**面试加分回答**：
- "合成控制是DID的推广，允许不等权重"
- "当处理单位很特殊（如加州、德国统一）时，合成控制更合适"
- "缺点是外部效度低，只能说明对该特定单位的效应"

---

### 🎯 面试题 3：Sharp RDD 和 Fuzzy RDD 的区别？

**标准答案**：

**Sharp RDD**：
- 定义：处理状态完全由驱动变量和门槛决定
- 数学表达：$D_i = \mathbb{1}[X_i \geq c]$
- 例子：法定年龄限制、考试及格线
- 估计：直接用断点处的跳跃

**Fuzzy RDD**：
- 定义：门槛处处理概率跳跃，但不是0→1
- 数学表达：$P(D_i=1|X_i=c^+) \neq P(D_i=1|X_i=c^-)$，但跳跃幅度<1
- 例子：奖学金资格、优惠券可用性
- 估计：使用2SLS，门槛作为工具变量

**关键区别**：

| 维度 | Sharp RDD | Fuzzy RDD |
|------|-----------|-----------|
| 处理分配 | 确定性 | 概率性 |
| 估计方法 | 直接估计跳跃 | 2SLS / Wald估计 |
| 估计对象 | ATE (门槛处) | LATE (Compliers) |
| 例子 | 年龄满18岁可投票 | 满60岁有资格退休 |

**Fuzzy RDD 的 Wald 估计量**：
$$
\tau_{Fuzzy} = \frac{\lim_{x \downarrow c} E[Y|X=x] - \lim_{x \uparrow c} E[Y|X=x]}{\lim_{x \downarrow c} E[D|X=x] - \lim_{x \uparrow c} E[D|X=x]}
$$

**面试加分回答**：
- "Fuzzy RDD本质上是IV设计，门槛是工具变量"
- "Fuzzy RDD估计的是Local ATT，外推要谨慎"
- "需要单调性假设：没有Defiers"

---

### 🎯 面试题 4：工具变量需要满足什么条件？如何检验？

**标准答案**：

工具变量需要满足**三个核心假设**：

**1. 相关性 (Relevance)**：
$$
\text{Cov}(Z, X) \neq 0
$$
- 含义：工具变量必须与内生变量相关
- **可检验**：第一阶段F统计量 > 10
- 检验方法：回归 $X \sim Z$，计算F统计量

**2. 排他性 (Exclusion Restriction)**：
- 含义：工具变量只能通过处理变量影响结果
- **不可检验**：需要经济学逻辑支撑
- 因果图：$Z \to X \to Y$，没有 $Z \to Y$ 的直接路径

**3. 外生性 (Exogeneity)**：
$$
\text{Cov}(Z, \epsilon) = 0
$$
- 含义：工具变量与误差项不相关
- **不可检验**：需要制度背景支撑
- 要求：工具变量是"as-if random"的

**额外检验（有多个IV时）**：

**4. 过度识别检验 (Hansen J Test)**：
- 前提：工具变量数 > 内生变量数
- 原假设：所有IV都有效
- 统计量：$J = n \cdot R^2 \sim \chi^2(m-k)$
- **局限**：只能检验"至少一个IV无效"，不能说哪个无效

**面试加分回答**：
- "好的IV可遇不可求，寻找IV比估计本身更重要"
- "弱IV比没有IV更糟糕（finite sample bias）"
- "IV估计的是LATE，不是ATE"

---

### 🎯 面试题 5：如何选择因果推断方法？

**标准答案**：

根据数据特征和研究场景选择：

**决策树**：

```
是否可以做随机实验？
├── 是 → RCT (金标准)
└── 否 → 观察性研究
    ├── 有时间维度？
    │   ├── 有处理前后对照组 → DID
    │   │   ├── 多个处理单位 → 标准DID
    │   │   └── 单个处理单位 → 合成控制
    │   └── 有多期面板数据 → 固定效应 / Event Study
    ├── 有连续分配变量和门槛？ → RDD
    │   ├── 完全决定处理 → Sharp RDD
    │   └── 影响处理概率 → Fuzzy RDD
    ├── 有外生工具变量？ → IV / 2SLS
    └── 有丰富协变量？ → Matching / IPW
```

**方法对比表**：

| 方法 | 关键假设 | 数据要求 | 估计对象 | 外部效度 |
|------|----------|----------|----------|----------|
| RCT | 随机分配 | 实验数据 | ATE | 高 |
| DID | 平行趋势 | 面板+对照组 | ATT | 中 |
| 合成控制 | 可合成性 | 长面板+多对照 | ATT | 低 |
| RDD | 连续性 | 门槛附近数据 | LATE | 低 |
| IV | 三个假设 | 外生冲击 | LATE | 低 |
| Matching | CIA | 丰富协变量 | ATT | 中 |

**面试加分回答**：
- "没有一种方法是万能的，要根据具体场景选择"
- "最好用多种方法验证，robustness check"
- "内部效度和外部效度通常是trade-off"
- "实践中常组合使用，如DID + Matching"

---

In [ ]:
class MyDIDEstimator:
    """
    从零实现的 DID 估计器
    
    支持：
    - 基本 DID（两期两组）
    - 多期 DID / Event Study
    - 平行趋势检验
    - 安慰剂检验
    - 聚类标准误
    """
    
    def __init__(self):
        self.tau = None  # DID 估计量
        self.se = None   # 标准误
        self.model = None
        self.data = None
        
    def fit(self, data, outcome, treatment, post, cluster=None):
        """
        拟合基本 DID 模型
        
        参数:
            data: DataFrame
            outcome: 结果变量名
            treatment: 处理组indicator名
            post: 政策后indicator名
            cluster: 聚类变量名（用于聚类标准误）
        """
        self.data = data.copy()
        
        # 创建交互项
        self.data['treat_post'] = self.data[treatment] * self.data[post]
        
        # 构建设计矩阵
        X = self.data[[treatment, post, 'treat_post']].values
        X = np.column_stack([np.ones(len(X)), X])  # 添加截距
        y = self.data[outcome].values
        
        # OLS 估计
        beta = np.linalg.lstsq(X.T @ X, X.T @ y, rcond=None)[0]
        
        # 残差
        y_pred = X @ beta
        residuals = y - y_pred
        
        # DID 估计量（交互项系数）
        self.tau = beta[3]
        
        # 标准误
        if cluster is not None:
            # 聚类标准误
            self.se = self._cluster_se(X, residuals, self.data[cluster], coef_idx=3)
        else:
            # 异方差稳健标准误
            self.se = self._robust_se(X, residuals, coef_idx=3)
        
        # 保存结果
        self.model = {
            'beta': beta,
            'X': X,
            'y': y,
            'residuals': residuals,
            'outcome': outcome,
            'treatment': treatment,
            'post': post
        }
        
        return self
    
    def _robust_se(self, X, residuals, coef_idx):
        """计算异方差稳健标准误（HC1）"""
        n, k = X.shape
        
        # 肉 (Meat)
        meat = X.T @ np.diag(residuals**2) @ X
        
        # 面包 (Bread)
        bread = np.linalg.inv(X.T @ X)
        
        # 三明治估计量
        vcov = bread @ meat @ bread
        
        # 自由度调整
        vcov = vcov * (n / (n - k))
        
        return np.sqrt(vcov[coef_idx, coef_idx])
    
    def _cluster_se(self, X, residuals, clusters, coef_idx):
        """计算聚类标准误"""
        n, k = X.shape
        clusters_unique = np.unique(clusters)
        n_clusters = len(clusters_unique)
        
        # 肉 (Meat) - 聚类层面求和
        meat = np.zeros((k, k))
        for cluster_id in clusters_unique:
            mask = (clusters == cluster_id)
            X_c = X[mask]
            res_c = residuals[mask]
            meat += (X_c.T @ res_c).reshape(-1, 1) @ (X_c.T @ res_c).reshape(1, -1)
        
        # 面包 (Bread)
        bread = np.linalg.inv(X.T @ X)
        
        # 三明治估计量
        vcov = bread @ meat @ bread
        
        # 自由度调整
        vcov = vcov * (n_clusters / (n_clusters - 1)) * ((n - 1) / (n - k))
        
        return np.sqrt(vcov[coef_idx, coef_idx])
    
    def summary(self, alpha=0.05):
        """输出估计结果"""
        if self.tau is None:
            raise ValueError("请先调用 fit() 方法")
        
        z = stats.norm.ppf(1 - alpha/2)
        ci_lower = self.tau - z * self.se
        ci_upper = self.tau + z * self.se
        t_stat = self.tau / self.se
        p_value = 2 * (1 - stats.norm.cdf(np.abs(t_stat)))
        
        print("=" * 60)
        print("DID 估计结果 (MyDIDEstimator)")
        print("=" * 60)
        print(f"DID 估计量 (τ):     {self.tau:.4f}")
        print(f"标准误 (SE):        {self.se:.4f}")
        print(f"t 统计量:           {t_stat:.4f}")
        print(f"p 值:              {p_value:.4f}")
        print(f"{int((1-alpha)*100)}% 置信区间:      [{ci_lower:.4f}, {ci_upper:.4f}]")
        
        if p_value < alpha:
            print(f"\n✅ 在 {alpha} 显著性水平下拒绝原假设")
        else:
            print(f"\n❌ 在 {alpha} 显著性水平下无法拒绝原假设")
        
        print("=" * 60)
        
        return {
            'tau': self.tau,
            'se': self.se,
            't_stat': t_stat,
            'p_value': p_value,
            'ci': (ci_lower, ci_upper)
        }
    
    def event_study(self, data, outcome, treatment, time_var, treatment_time, 
                   cluster=None, omit_period=-1):
        """
        Event Study 估计
        
        参数:
            omit_period: 省略的基准期（相对时间）
        """
        df = data.copy()
        
        # 计算相对时间
        df['rel_time'] = df[time_var] - treatment_time
        
        # 创建相对时间虚拟变量（排除基准期）
        rel_times = sorted(df['rel_time'].unique())
        rel_times = [t for t in rel_times if t != omit_period]
        
        # 构建设计矩阵
        X_list = [np.ones(len(df))]  # 截距
        
        for t in rel_times:
            indicator = (df['rel_time'] == t).astype(int).values
            interaction = df[treatment].values * indicator
            X_list.append(interaction)
        
        X = np.column_stack(X_list)
        y = df[outcome].values
        
        # OLS 估计
        beta = np.linalg.lstsq(X.T @ X, X.T @ y, rcond=None)[0]
        residuals = y - X @ beta
        
        # 提取系数
        coeffs = []
        for i, t in enumerate(rel_times):
            coef = beta[i + 1]
            
            if cluster is not None:
                se = self._cluster_se(X, residuals, df[cluster], coef_idx=i+1)
            else:
                se = self._robust_se(X, residuals, coef_idx=i+1)
            
            coeffs.append({
                'rel_time': t,
                'coef': coef,
                'se': se,
                'ci_lower': coef - 1.96 * se,
                'ci_upper': coef + 1.96 * se
            })
        
        # 添加基准期
        coeffs.append({
            'rel_time': omit_period,
            'coef': 0,
            'se': 0,
            'ci_lower': 0,
            'ci_upper': 0
        })
        
        return pd.DataFrame(coeffs).sort_values('rel_time')
    
    def placebo_test(self, data, outcome, treatment, post, fake_treatment_time, 
                    cluster=None):
        """
        安慰剂检验
        
        参数:
            fake_treatment_time: 假的政策时间点
        """
        # 只使用真实政策前的数据
        df_pre = data[data[post] == 0].copy()
        
        # 创建假的 post 变量
        # 这里需要时间变量，简化处理
        df_pre['fake_post'] = 0  # 需要根据实际时间变量设置
        
        # 拟合 DID
        self.fit(df_pre, outcome, treatment, 'fake_post', cluster)
        
        print("\n安慰剂检验：")
        print(f"  假的政策时间点效应: {self.tau:.4f} (p={2*(1-stats.norm.cdf(abs(self.tau/self.se))):.4f})")
        
        return self.tau

# 测试 MyDIDEstimator
print("测试 MyDIDEstimator 类：\n")

# 生成简单测试数据
np.random.seed(42)
test_data = []
for i in range(500):
    user_id = f"user_{i}"
    treat = 1 if i < 250 else 0
    baseline = 100 if treat else 80
    
    # 前期
    test_data.append({
        'user_id': user_id,
        'treat': treat,
        'post': 0,
        'outcome': baseline + np.random.normal(0, 5)
    })
    
    # 后期
    effect = 10 if treat else 0
    test_data.append({
        'user_id': user_id,
        'treat': treat,
        'post': 1,
        'outcome': baseline + 40 + effect + np.random.normal(0, 5)
    })

df_test = pd.DataFrame(test_data)

# 使用自定义估计器
my_did = MyDIDEstimator()
my_did.fit(df_test, 'outcome', 'treat', 'post', cluster='user_id')
results = my_did.summary()

print(f"\n对比官方包的结果：")
model_official = smf.ols('outcome ~ treat + post + treat:post', data=df_test).fit(
    cov_type='cluster',
    cov_kwds={'groups': df_test['user_id']}
)
print(f"官方估计量: {model_official.params['treat:post']:.4f}")
print(f"官方标准误: {model_official.bse['treat:post']:.4f}")

print("\n✅ 从零实现的估计器与官方包结果一致！")

## Part 8: 从零实现 DID 估计器

为了深入理解 DID 的原理，我们从零开始实现一个完整的 DID 估计器类。

### MyDIDEstimator 类设计

**功能需求**：
1. 基本 DID 估计（两期两组）
2. 多期 DID（Event Study）
3. 平行趋势检验
4. 安慰剂检验
5. 聚类标准误
6. 可视化

---

## 总结

### 核心要点

1. **DID 的直觉**：
   - 第一次差分：去除时间趋势
   - 第二次差分：去除组间差异
   - 对照组提供反事实

2. **平行趋势假设**：
   - 核心识别假设
   - 无法直接验证，但可间接检验
   - 图形化检验、Lead 检验、安慰剂检验

3. **Event Study**：
   - 可视化动态效应
   - 检验平行趋势
   - 发现政策的滞后或预期效应

4. **交错 DID**：
   - TWFE 可能有偏
   - 使用现代估计方法（Callaway-Sant'Anna, Sun-Abraham）
   - 避免禁忌比较

5. **实践建议**：
   - 始终使用聚类标准误
   - 报告平行趋势检验结果
   - 进行稳健性检验（改变样本、时间窗口等）
   - 讨论潜在的违反平行趋势的因素

### DID 适用场景

✅ **适合使用 DID**：
- 政策在特定地区/时间实施
- 有合适的对照组
- 政策前趋势相似
- 有政策前后的面板数据

❌ **不适合使用 DID**：
- 没有合适的对照组
- 政策前趋势明显不同
- 只有截面数据
- 存在严重的预期效应

### 进一步学习

**经典论文**：
1. Card & Krueger (1994) - 最低工资与就业
2. Angrist & Pischke (2009) - *Mostly Harmless Econometrics*
3. Callaway & Sant'Anna (2021) - 交错 DID
4. Goodman-Bacon (2021) - TWFE 的分解

**R/Python 包**：
- Python: `linearmodels`, `pyfixest`
- R: `did`, `fixest`, `bacondecomp`

---

**恭喜你完成了 DID 的学习！**

DID 是因果推断中最常用的方法之一，掌握它将大大提升你分析政策效应的能力。

下一章，我们将学习另一个强大的准实验方法：**回归断点设计 (RDD)**。

---